# RAG Evaluation Notebook

In [5]:
import os
import sys

# Fix import path (same as dataset creation)
project_root = os.path.abspath('../..')
sys.path.insert(0, project_root)

from evaluations.runners.rag_evaluator import RAGEvaluator
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np


print("✅ Imports successful!")

✅ Imports successful!


In [7]:
# Initialize evaluator
evaluator = RAGEvaluator()
print(f"Results will be saved to: {evaluator.results_dir}")

# Find the most recent dataset files
dataset_dir = Path("../datasets/rag_evaluation")
chunks_files = list(dataset_dir.glob("chunks_*.json"))
queries_files = list(dataset_dir.glob("queries_*.json"))

if chunks_files and queries_files:
    # Use the most recent files (by filename)
    chunks_file = str(sorted(chunks_files)[-1])
    queries_file = str(sorted(queries_files)[-1])
    
    print(f"Using dataset files:")
    print(f"📁 Chunks: {chunks_file}")
    print(f"📁 Queries: {queries_file}")
    
    # Load to check dataset size
    with open(chunks_file, 'r') as f:
        chunks_data = json.load(f)
    with open(queries_file, 'r') as f:
        queries_data = json.load(f)
        
    print(f"\nDataset info:")
    print(f"  Total chunks: {len(chunks_data)}")
    print(f"  Total queries: {len(queries_data)}")
else:
    print("❌ No dataset files found. Please run dataset_creation.ipynb first.")
    chunks_file = None
    queries_file = None

Results will be saved to: evaluations/results/rag_evaluation
Using dataset files:
📁 Chunks: ../datasets/rag_evaluation/chunks_seattle_100listings_20250529_204239.json
📁 Queries: ../datasets/rag_evaluation/queries_seattle_100listings_20250529_204239.json

Dataset info:
  Total chunks: 250
  Total queries: 596
